In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def read_and_preprocess(filepath):
    
    # initial reading
    data = pd.read_csv(filepath, index_col=0)
    
    # move species from index to the table
    data["species"] = data.index
    
    # create numeric idex
    data.index = range(len(data))
    
    # create categorical feature from Phylum (string)
    data["Phylum_Numeric"] = LabelEncoder().fit_transform(data["Phylum"].tolist())
    
    # remove features we will not use
    data = data.drop(["Phylum", "species", "occurrences", "NoSpecies", 
                      "C_Cnumeric", "SC_Numeric", "MaxD_Numeric", "System_Numeric"], axis=1)
    
    # create features and target dataframes
    features = data.drop(["extinct"], axis=1)
    target = data["extinct"]
    
    # create lists with categorical and continious features' names
    continious_cols = []
    categorical_cols = features.columns.drop(continious_cols).tolist()
    
    # make list of indexes
    categorical_idx = [features.columns.tolist().index(col) for col in categorical_cols]
        
    return features, target, categorical_idx

In [3]:
def model_me(filepath):
    
    # prepare data for modeling
    features, target, cat_idx = read_and_preprocess(filepath)
    
    # create cross-validation instance
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    
    # create holders for scores and feature importances
    cv_scores = []
    cv_roc_curves = []
    feature_importances = []
    
    # loop over different validation splits and save results
    for i, (train_idx, test_idx) in enumerate(cv.split(features, target)):
        
        # Gradient boosting model instance
        model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
        
        # Create data classes for train/test
        train_features = features.iloc[train_idx, :]
        train_target = target[train_idx]
        
        test_features = features.iloc[test_idx, :]
        test_target = target[test_idx]
        
        # Train model
        model.fit(train_features, train_target)
        
        # save score from the individual split
        cv_scores.append(roc_auc_score(test_target, model.predict_proba(test_features)[:, 1]))
        
        # save roc curve ordinates from the individual split
        cv_roc_curves.append(roc_curve(test_target, model.predict_proba(test_features)[:, 1], 
                                       drop_intermediate=False))
        
        # save feature importances from the individual split
        feature_importances.append(model.feature_importances_ * 100)
        
        # save model itself
        pickle.dump(model, open(f"../results/models/RFC_{filepath[8:-4]}_Split{i+1}", 'wb'))
    
    cv_scores = pd.DataFrame(np.array(cv_scores),
                             index=["Split1", "Split2", "Split3", "Split4", "Split5"],
                             columns=["AUC"])
    
    cv_roc_curves = {cv_scores.index[i]: {"FPR": cv_roc_curves[i][0], 
                                          "TPR": cv_roc_curves[i][1]} for i in range(len(cv_roc_curves))}
    
    feature_importances = pd.DataFrame(np.array(feature_importances), 
                                       index=["Split1", "Split2", "Split3", "Split4", "Split5"], 
                                       columns=features.columns)
    
    return cv_scores, cv_roc_curves, feature_importances

In [4]:
%%time

# loop over individual Time Intervals
for i in range(1, 5):
    
    # obtain scores and feature importances using cross-validation
    cv, rc, fi = model_me(f"../data/TimeInterval{i}.csv")
    
    cv.to_csv(f"../results/RFC_TimeInterval{i}_AUC.csv")
    fi.to_csv(f"../results/RFC_TimeInterval{i}_FI.csv")
    
    np.save(f"../results/RFC_TimeInterval{i}_RC.npy", rc)
    
    print(cv.mean())
    print(fi.mean())

AUC    0.724477
dtype: float64
K_Numeric          4.420257
Min_Numeric        9.660984
C_Numeric         10.806933
S_Numeric          5.832947
O_Numeric         11.386313
T_Numeric          5.329486
M_Numeric          4.862513
R_Numeric          2.999791
Re_Numeric         4.060277
MinD_Numeric      29.499098
Phylum_Numeric    11.141400
dtype: float64
AUC    0.744212
dtype: float64
K_Numeric          3.888474
Min_Numeric        8.588218
C_Numeric         12.943681
S_Numeric          7.439172
O_Numeric         10.044457
T_Numeric          6.276083
M_Numeric          5.693095
R_Numeric          8.601832
Re_Numeric         3.980825
MinD_Numeric      24.572106
Phylum_Numeric     7.972057
dtype: float64
AUC    0.772793
dtype: float64
K_Numeric          8.431457
Min_Numeric       11.770667
C_Numeric          8.533356
S_Numeric          7.843410
O_Numeric         11.405629
T_Numeric          5.957646
M_Numeric          9.128803
R_Numeric          3.832569
Re_Numeric         4.636032
MinD_Nume